In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
from sklearn.feature_extraction import FeatureHasher
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from tqdm import tqdm
from scipy.sparse import csc_matrix, csr_matrix, hstack
import time
import numpy as np
import pickle

In [2]:
train = pd.read_pickle(FILE.train_ori.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_ori.value)
print('test shape is: {}'.format(test.shape))

# X = pd.concat([train.drop(['click'],axis=1),test])
X = pd.concat([train,test],sort=False)
print(X.shape)


X_shiyi = pd.read_pickle(FILE.shiyi_fillna_ori.value)
print(X_shiyi.shape)

X = X.merge(X_shiyi[['time_hour','instance_id']],how='inner',on='instance_id')
print(X.shape)
ignore_columns = ['instance_id','time','click'] + ['creative_is_js', 'creative_is_voicead', 'app_paid']

train shape is: (1001650, 35)
test shape is: (40024, 34)
(1041674, 35)
(1041674, 45)
(1041674, 36)


In [3]:
need_process_col = list(set(X.columns) - set(ignore_columns))
X_ = X[need_process_col].copy()
matts = []
counter = 0
for col in tqdm(X_.columns):
#     X_[col] = le.fit_transform(X_[col].astype(str))
    X_[col] = X_[col].astype(str)
    nunique = X_[col].nunique()
    n_fe = int(np.log2(nunique)/np.log2(1.2))
    if n_fe < 2:
        n_fe = 2
    h = FeatureHasher(n_features=n_fe,input_type="string")
    mat = h.fit_transform(X_[col])
    matts.append(csc_matrix(mat))

100%|██████████| 30/30 [01:33<00:00,  3.12s/it]


In [4]:
t1 = time.time()
X_matrix = hstack(matts)
print(X_matrix.shape)
print(time.time()-t1)

(1041674, 671)
0.17454862594604492


In [5]:
pickle.dump(X_matrix,open(FILE.X_fe_hashing_format.value.format('col{}_hasUtag'.format(X_matrix.shape[1])),'wb'))